In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#from bs4 import BeautifulSoup
from glob import glob
#import linkcheck
import os
import pandas as pd
from subprocess import run
from tqdm import tqdm

from property_scraper import convert_hyperlink_rel2global, fix_hyperlinks, get_filename_from_identifier
from property_scraper.scraper import Scraper

In [3]:
rc_filename = os.path.join(os.path.expanduser('~'), 'property_scraper.json')

In [4]:
scraper = Scraper()

In [5]:
scraper.configuration = scraper.read_configuration(rc_filename)
#scraper.show_configuration()

JSONDecodeError: Expecting property name enclosed in double quotes: line 281 column 9 (char 16791)

In [56]:
print(scraper.configuration['folder'])
print(scraper.configuration['output folder'])

/home/data/property_scraper/demos/downloads
/home/www/html/italia.estate-database.com


In [57]:
!ls ../data/*.txt

ls: cannot access '../data/*.txt': No such file or directory


In [58]:
filename = '../data/spiders.txt'

In [59]:
folder = '/home/data/property_scraper/demos/downloads'
spiders = sorted([os.path.relpath(x, folder) for x in glob(os.path.join(folder, '*')) if os.path.isdir(x)])
print('The following scrapy spider searches are available:\n  - ' + '\n  - '.join(spiders))

The following scrapy spider searches are available:
  - astagiudiziaria
  - astainsieme
  - astalegale
  - asteavvisi
  - astegiudiziarie
  - asteimmobili
  - astetelematiche
  - astetribunalelaspezia
  - astetribunaleravenna
  - astetribunalesavona
  - blinks
  - canaleaste
  - casa
  - entietribunali
  - fallcoaste
  - gateaway
  - guber
  - idealista
  - immobiliallasta
  - immobiliare
  - intribunale
  - intrum
  - legalmente
  - lionard
  - portaleaste
  - pvp
  - spazioaste
  - tecnocasa
  - trovoaste
  - trovocasa


In [60]:
if 0 == 1:
    with open(filename, 'r') as f:
        spiders = sorted([x.strip() for x in f.readlines() if not x.startswith('#')])
    print('The following scrapy spiders are available:\n\t' + '\n\t'.join(spiders))

In [61]:
spider = 'astagiudiziaria'

In [62]:
print(scraper.configuration[spider])

{'root': 'https://www.astagiudiziaria.com'}


In [63]:
rootname = scraper.configuration[spider]['root']
print(rootname)

https://www.astagiudiziaria.com


In [67]:
filenames = sorted(glob(os.path.join(folder, spider, f'{spider}_*.html')))
print(f'There are {len(filenames)} files for this spider.')
filenames[:10]

There are 660 files for this spider.


['/home/data/property_scraper/demos/downloads/astagiudiziaria/astagiudiziaria_20230321015034_00.html',
 '/home/data/property_scraper/demos/downloads/astagiudiziaria/astagiudiziaria_20230321015045_00.html',
 '/home/data/property_scraper/demos/downloads/astagiudiziaria/astagiudiziaria_20230321015056_00.html',
 '/home/data/property_scraper/demos/downloads/astagiudiziaria/astagiudiziaria_20230321015107_00.html',
 '/home/data/property_scraper/demos/downloads/astagiudiziaria/astagiudiziaria_20230321015118_00.html',
 '/home/data/property_scraper/demos/downloads/astagiudiziaria/astagiudiziaria_20230321015128_00.html',
 '/home/data/property_scraper/demos/downloads/astagiudiziaria/astagiudiziaria_20230321015139_00.html',
 '/home/data/property_scraper/demos/downloads/astagiudiziaria/astagiudiziaria_20230321015150_00.html',
 '/home/data/property_scraper/demos/downloads/astagiudiziaria/astagiudiziaria_20230321020432_00.html',
 '/home/data/property_scraper/demos/downloads/astagiudiziaria/astagiudizi

In [71]:
pbar = tqdm(filenames, position=0)
filenames = ['/home/data/property_scraper/demos/downloads/astagiudiziaria/astagiudiziaria_20230321015045_00.html']
for filename in pbar:
    with open(filename, 'r') as f:
        content = f.read()
        soup = convert_hyperlink_rel2global(content, rootname)
        basename = os.path.basename(filename)
        ofilename = os.path.join(scraper.configuration['output folder'], spider, basename)
        pbar.set_description(f'{basename}')
        with open(ofilename, 'wb') as f:
            f.write(soup.prettify("utf-8"))
        break

astagiudiziaria_20230321015034_00.html:   0%|                                                                                                                                                                       | 0/660 [00:00<?, ?it/s]

/convegni
https://www.astagiudiziaria.com/convegni
/guida
https://www.astagiudiziaria.com/guida
/servizi
https://www.astagiudiziaria.com/servizi
/bollettini/search?type=i&title=Bollettino+Immobiliare
https://www.astagiudiziaria.com/bollettini/search?type=i&title=Bollettino+Immobiliare
/bollettini/search?type=m&title=Bollettino+Mobiliare
https://www.astagiudiziaria.com/bollettini/search?type=m&title=Bollettino+Mobiliare
/myAsta/login
https://www.astagiudiziaria.com/myAsta/login
/reserved/login
https://www.astagiudiziaria.com/reserved/login
/
https://www.astagiudiziaria.com/
/
https://www.astagiudiziaria.com/
/categoria/immobili
https://www.astagiudiziaria.com/categoria/immobili
/categoria/mobili
https://www.astagiudiziaria.com/categoria/mobili
/categoria/aziende
https://www.astagiudiziaria.com/categoria/aziende
/categoria/altro
https://www.astagiudiziaria.com/categoria/altro
/categoria/valoricrediti
https://www.astagiudiziaria.com/categoria/valoricrediti
/categoria/immobili
https://www.

In [6]:
script_filename = '../demos/run_spiders.bash'
with open(script_filename, 'w') as f:
    f.write('#!/bin/bash\n\n')
    for spider in spiders:
        folder = f"../www/{spider}"
        if not os.path.exists(folder):
            os.makedirs(folder)

        command = f"echo scrapy runspider ../property_scraper/{spider}/spiders/{spider}.py --output ../demos/{spider}.csv"
        f.write(command + '\n')
        #run(command, shell=True, check=True) #, bufsize=bufsize, stdin=PIPE, stdout=PIPE, close_fds=True)
    f.write('')

In [7]:
stop here

SyntaxError: invalid syntax (4067800170.py, line 1)

In [13]:
url = 'https://www.atico.es/resultados-de-la-busqueda/page/2/'


'resultados-de-la-busqueda'

In [11]:
search_filename = '../demos/alltownri.csv'
properties = pd.read_csv(search_filename)
display(properties.head(3).T)

,0,1,2
home_listing_id,1331326,NaN,NaN
name,"100 Monticello Road, Pawtucket",NaN,NaN
availability,for_sale,for_sale,for_sale
address.addr1,100 Monticello Road,NaN,NaN
address.city,Pawtucket,NaN,NaN
address.region,Rhode Island,NaN,NaN
address.country,United States,United States,United States
address.postal_code,02861,NaN,NaN
image[0].url,https://t3.realgeeks.media/thumbnail/m0Ppx5fJw...,NaN,NaN
price,"$325,000",NaN,NaN


In [ ]:
name = 'immobiliare'

In [ ]:
s = 'https://www.idealista.it/vendita-case/ravenna-ravenna/'
s.split('/')[4]

In [ ]:
s = 'https://www.immobiliare.it/vendita-case/ravenna-provincia/?criterio=rilevanza'
s.split('/')[4]

### PVP

In [ ]:
#filenames = glob('/home/git/property_scraper/www/pvp/pvp_LTT*.html')
#_filenames = glob('/home/git/property_scraper/www/pvp/pvp_*.html')
#filenames = [x for x in _filenames if 'pvp_LTT' not in x]
filenames = glob(f'/home/git/property_scraper/www/{name}/{name}_*.html')
print(len(filenames))

In [ ]:
func = {'https://pvp.giustizia.it/pvp/it/dettaglio_annuncio.page?': get_filename_from_identifier}
keys = ['contentId']
root = 'http://localhost:8000/{name}/{name}'

In [ ]:
pbar = tqdm(list(reversed(filenames)), position=0)
for filename in pbar:
    fix_hyperlinks(filename, func, keys, root, True)

### Intrum

In [5]:
name = 'intrum'
#filenames = glob('/home/git/property_scraper/www/pvp/pvp_LTT*.html')
#_filenames = glob('/home/git/property_scraper/www/pvp/pvp_*.html')
#filenames = [x for x in _filenames if 'pvp_LTT' not in x]
filenames = glob(f'/home/git/property_scraper/www/{name}/{name}_*.html')
print(len(filenames))

1


In [6]:
func = {}
keys = []
root = 'http://localhost:8000/{name}/{name}'

In [7]:

pbar = tqdm(filenames, position=0)

for filename in pbar:
    with open(filename, 'r') as f:
        content = f.read()
        soup = convert_hyperlink_rel2global(content, rootname)
    ofilename = filename
    with open(ofilename, 'wb') as f:
        f.write(soup.prettify("utf-8"))        

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.33it/s]


In [ ]:
#linkcheck.LinkChecker

In [ ]:
#linkcheck.LinkChecker